In [2]:
import os
import sys
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

# Set up path to project root
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

# Custom modules
from src.problems.ising import relaxed_ising_energy, grad_relaxed_ising, ising_energy
from src.optimizers.sa import sa_continuous, sa_discrete
from src.optimizers.gd import gradient_descent
from src.utils.utils_experiments import bootstrap_experiment

# Output directory
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Hyperparameter grids
sa_grid = {
    'T0': [10, 50, 100],
    'alpha': [0.95, 0.99, 0.995],
    'step_size': [0.001, 0.01, 0.05, 0.1, 0.3]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1]
}

lattice_shape = (10, 10)
num_runs = 20

In [32]:
def run_one_sa_discrete(T0, alpha, num_runs=20, hamming_threshold=0):
    print(f"SA-discrete: T0={T0}, alpha={alpha}")
    result = bootstrap_experiment(
        sa_discrete,
        num_runs,               # <- positional
        ising_energy,           # <- positional
        lattice_size=lattice_shape,
        T_init=T0,
        alpha=alpha,
        max_iter=50000,
        tol=1e-6,
        is_discrete=True
    )       


    # compute energy stats from final values
    final_values = np.array(result["final_values"])
    final_states = result["final_states"]

    stats = {
        "mean": np.mean(final_values),
        "best": np.min(final_values),
        "worst": np.max(final_values),
        "std": np.std(final_values)
    }

    # identify best state (lowest energy)
    best_idx = np.argmin(final_values)
    best_state = final_states[best_idx]

    # compute Hamming-based near-optimal count
    near_optimal_count = sum(
        1 for state in final_states if np.sum(state != best_state) <= hamming_threshold
    )

    return {
        "T0": T0,
        "alpha": alpha,
        "mean": stats["mean"],
        "best": stats["best"],
        "worst": stats["worst"],
        "std": stats["std"],
        "near_optimal_count": near_optimal_count,
        "hamming_threshold": hamming_threshold
    }


def grid_search_sa_discrete(grid, num_runs=20, hamming_threshold=0):
    results = []
    for i, (T0, alpha) in enumerate(tqdm(itertools.product(grid["T0"], grid["alpha"]))):
        print(f"SA-discrete: {i+1}/{len(grid['T0']) * len(grid['alpha'])} combinations", flush=True)
        res = run_one_sa_discrete(T0, alpha, num_runs=num_runs, hamming_threshold=hamming_threshold)
        results.append(res)
    return pd.DataFrame(results)

In [33]:
def wrap_relaxed_ising(f):
    return lambda x: f(x.reshape(lattice_shape))

def run_one_sa_continuous(f, T0, alpha, step_size, num_runs=20):
    dim = lattice_shape[0] * lattice_shape[1]
    result = bootstrap_experiment(
        sa_continuous, num_runs, f,
        x_init=None, bounds=[(-1, 1)] * dim,
        T0=T0, alpha=alpha, step_size=step_size,
        tol=1e-6, max_iter=20000
    )
    stats = result["stats"]
    return {
        "T0": T0, "alpha": alpha, "step_size": step_size,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"],
        "std": stats["std"], "epsilon": stats["epsilon"],
        "near_optimal_count": stats["near_optimal_count"]
    }

def grid_search_sa_continuous(f, grid, num_runs=20):
    results = []
    combinations = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    for i, (T0, alpha, step_size) in enumerate(tqdm(combinations)):
        print(f"SA-continuous: {i+1}/{len(combinations)} combinations")
        res = run_one_sa_continuous(f, T0, alpha, step_size, num_runs)
        results.append(res)
    return pd.DataFrame(results)


def run_one_gd_continuous(f, grad, lr, num_runs=20):
    dim = lattice_shape[0] * lattice_shape[1]
    result = bootstrap_experiment(
        gradient_descent, num_runs, f, grad,
        x_init=None, lr=lr, max_iter=50000, tol=1e-6
    )
    stats = result["stats"]
    return {
        "lr": lr,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"],
        "std": stats["std"], "epsilon": stats["epsilon"],
        "near_optimal_count": stats["near_optimal_count"]
    }

def grid_search_gd_continuous(f, grad, grid, num_runs=20):
    results = []
    for i, lr in enumerate(tqdm(grid["lr"])):
        print(f"GD: {i+1}/{len(grid['lr'])} combinations")
        res = run_one_gd_continuous(f, grad, lr, num_runs)
        results.append(res)
    return pd.DataFrame(results)


# Running actual Gridsearch

In [ ]:
# Discrete Ising (SA only)
print("\nRunning SA-discrete on ising_energy")
df_sa_discrete = grid_search_sa_discrete(sa_grid, num_runs)
df_sa_discrete.to_csv(os.path.join(results_dir, "gridsearch_sa_discrete_ising.csv"), index=False)

# Relaxed Ising (SA)
print("\nRunning SA-continuous on relaxed_ising")
relaxed_f = wrap_relaxed_ising(relaxed_ising_energy)
df_sa_continuous = grid_search_sa_continuous(relaxed_f, sa_grid, num_runs)
df_sa_continuous.to_csv(os.path.join(results_dir, "gridsearch_sa_continuous_ising.csv"), index=False)

# Relaxed Ising (GD)
print("\nRunning GD on relaxed_ising")
grad_wrapped = lambda x: grad_relaxed_ising(x.reshape(lattice_shape)).flatten()
df_gd = grid_search_gd_continuous(relaxed_f, grad_wrapped, gd_grid, num_runs)
df_gd.to_csv(os.path.join(results_dir, "gridsearch_gd_ising.csv"), index=False)



Running SA-discrete on ising_energy


0it [00:00, ?it/s]

SA-discrete: 1/9 combinations
SA-discrete: T0=10, alpha=0.95
Run 1/20...
Run 2/20...
Run 3/20...
Run 4/20...
Run 5/20...
Run 6/20...
Run 7/20...
Run 8/20...
Run 9/20...
Run 10/20...
Run 11/20...
Run 12/20...
Run 13/20...
Run 14/20...
Run 15/20...
Run 16/20...
Run 17/20...
Run 18/20...
Run 19/20...
Run 20/20...


1it [00:05,  5.91s/it]

SA-discrete: 2/9 combinations
SA-discrete: T0=10, alpha=0.99
Run 1/20...
Run 2/20...
Run 3/20...
Run 4/20...
Run 5/20...
Run 6/20...
Run 7/20...
Run 8/20...
Run 9/20...
Run 10/20...
Run 11/20...
Run 12/20...
Run 13/20...
Run 14/20...
Run 15/20...
Run 16/20...
Run 17/20...
Run 18/20...
Run 19/20...
Run 20/20...


2it [00:14,  7.43s/it]

SA-discrete: 3/9 combinations
SA-discrete: T0=10, alpha=0.995
Run 1/20...
Run 2/20...
Run 3/20...
Run 4/20...
Run 5/20...
Run 6/20...
Run 7/20...
Run 8/20...
Run 9/20...
Run 10/20...
Run 11/20...
Run 12/20...
Run 13/20...
